In [2]:
import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from types import SimpleNamespace
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# 导入用户的模块
from data_provider.data_factory import data_provider
from exp.exp_basic import Exp_Basic
from models import SparseTSF
from utils.tools import EarlyStopping, adjust_learning_rate, visual, test_params_flop
from utils.metrics import metric


# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


SyntaxError: invalid syntax (3374803015.py, line 18)

In [2]:
class SimpleETTDataset(Dataset):
    def __init__(self, data, seq_len, pred_len):
        """
        data: numpy array, shape [time, features]
        seq_len: 输入序列长度
        pred_len: 预测序列长度
        """
        self.data = data
        self.seq_len = seq_len
        self.pred_len = pred_len

    def __len__(self):
        return len(self.data) - self.seq_len - self.pred_len

    def __getitem__(self, index):
        x = self.data[index : index + self.seq_len, :]
        y = self.data[index + self.seq_len : index + self.seq_len + self.pred_len, :]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


In [12]:
import torch
import torch.nn as nn


class Model(nn.Module):
    def __init__(self, configs):
        super(Model, self).__init__()

        # get parameters
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.enc_in = configs.enc_in
        self.period_len = configs.period_len

        self.seg_num_x = self.seq_len // self.period_len
        self.seg_num_y = self.pred_len // self.period_len

        self.conv1d = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=1 + 2 * (self.period_len // 2),
                                stride=1, padding=self.period_len // 2, padding_mode="zeros", bias=False)

        self.linear = nn.Linear(self.seg_num_x, self.seg_num_y, bias=False)


    def forward(self, x):
        batch_size = x.shape[0]
        # normalization and permute     b,s,c -> b,c,s
        seq_mean = torch.mean(x, dim=1).unsqueeze(1)
        x = (x - seq_mean).permute(0, 2, 1)

        # 1D convolution aggregation
        x = self.conv1d(x.reshape(-1, 1, self.seq_len)).reshape(-1, self.enc_in, self.seq_len) + x

        # downsampling: b,c,s -> bc,n,w -> bc,w,n
        x = x.reshape(-1, self.seg_num_x, self.period_len).permute(0, 2, 1)

        # sparse forecasting
        y = self.linear(x)  # bc,w,m

        # upsampling: bc,w,m -> bc,m,w -> b,c,s
        y = y.permute(0, 2, 1).reshape(batch_size, self.enc_in, self.pred_len)

        # permute and denorm
        y = y.permute(0, 2, 1) + seq_mean

        return y


In [13]:
def train_model(model, train_loader, val_loader, config, device):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=config.learning_rate, 
        steps_per_epoch=len(train_loader), epochs=config.train_epochs
    )
    best_val_loss = float('inf')
    patience = config.patience
    wait = 0

    for epoch in range(config.train_epochs):
        model.train()
        train_loss = 0.0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.item() * x.size(0)
        train_loss /= len(train_loader.dataset)

        # 验证
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                output_val = model(x_val)
                loss_val = criterion(output_val, y_val)
                val_loss += loss_val.item() * x_val.size(0)
        val_loss /= len(val_loader.dataset)

        print(f"Epoch [{epoch+1}/{config.train_epochs}]: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

        # 保存最优模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), config.best_model_path)
            print("Best model saved.")
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping triggered.")
                break

def test_model(model, test_loader, device, best_model_path):
    # 加载最优模型
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model.eval()
    criterion = nn.MSELoss()
    test_loss = 0.0
    mae = 0.0
    with torch.no_grad():
        for x_test, y_test in test_loader:
            x_test, y_test = x_test.to(device), y_test.to(device)
            output = model(x_test)
            loss = criterion(output, y_test)
            test_loss += loss.item() * x_test.size(0)
            mae += torch.mean(torch.abs(output - y_test)).item() * x_test.size(0)
    test_loss /= len(test_loader.dataset)
    mae /= len(test_loader.dataset)
    print(f"Test Loss (MSE): {test_loss:.4f}, Test MAE: {mae:.4f}")
    return test_loss, mae


In [14]:
def load_data(filename):
    """
    从CSV文件中加载数据，假设第一列是日期，可选。
    """
    df = pd.read_csv(filename)
    if 'date' in df.columns:
        df = df.drop(columns=['date'])
    data = df.values  # [time, features]
    return data


In [15]:
# 定义路径和参数
root_path_name = './'
data_path_name_train = 'ETTh2.csv'  # 用于训练的ETTh1数据集
data_path_name_test = 'ETTh1.csv'   # 用于测试的ETTh2数据集

# 检查并创建日志目录
log_dir = './logs'
os.makedirs(log_dir, exist_ok=True)

model_name = 'SparseTSF'

seq_len = 720
pred_len_list = [96, 192, 336, 720]

# 加载训练和测试数据
data_ETTh1 = load_data(os.path.join(root_path_name, data_path_name_train))
data_ETTh2 = load_data(os.path.join(root_path_name, data_path_name_test))

# 标准化：使用训练集的均值和标准差
mean = data_ETTh1.mean(axis=0)
std = data_ETTh1.std(axis=0)
data_ETTh1 = (data_ETTh1 - mean) / std
data_ETTh2 = (data_ETTh2 - mean) / std  # 使用相同的均值和标准差

# 划分训练集和验证集（在ETTh1上）
n = len(data_ETTh1)
train_ratio = 0.7
val_ratio = 0.1
train_end = int(n * train_ratio)
val_end = int(n * (train_ratio + val_ratio))

train_data = data_ETTh1[:train_end]
val_data = data_ETTh1[train_end:val_end]

# 创建Dataset和DataLoader
batch_size = 256

def create_dataloaders(train_data, val_data, test_data, seq_len, pred_len, batch_size):
    train_dataset = SimpleETTDataset(train_data, seq_len, pred_len)
    val_dataset = SimpleETTDataset(val_data, seq_len, pred_len)
    test_dataset = SimpleETTDataset(test_data, seq_len, pred_len)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader

# 定义一个字典来存储每个 pred_len 的结果
results = {}

In [16]:
for pred_len in pred_len_list:
    print(f"\n{'='*20} Training with pred_len = {pred_len} {'='*20}\n")
    
    # 创建对应的 DataLoader
    train_loader, val_loader, test_loader = create_dataloaders(train_data, val_data, data_ETTh2, seq_len, pred_len, batch_size)
    
    # 配置参数
    config = SimpleNamespace(
        seq_len=seq_len,
        pred_len=pred_len,
        enc_in=data_ETTh1.shape[1],
        period_len=24,
        d_model=64,
        model_type='graph',
        learning_rate=0.03,
        train_epochs=30,
        patience=5,
        best_model_path=f"./logs/{model_name}_ETTh1_seq{seq_len}_pred{pred_len}.pth"
    )
    
    # 初始化模型
    model = Model(config).to(device)
    
    # 训练模型
    start_time = time.time()
    train_model(model, train_loader, val_loader, config, device)
    print(f"Training completed in {time.time() - start_time:.2f} seconds.")
    
    # 测试模型在ETTh2上的泛化性能
    print(f"\nTesting on ETTh2 with pred_len = {pred_len}")
    test_loss, test_mae = test_model(model, test_loader, device, config.best_model_path)
    
    # 存储结果
    results[pred_len] = {
        'Test MSE': test_loss,
        'Test MAE': test_mae
    }



==================== Training with pred_len = 96 ====================

Epoch [1/30]: Train Loss = 0.6746, Val Loss = 0.3296
Best model saved.
Epoch [2/30]: Train Loss = 0.4223, Val Loss = 0.2935
Best model saved.
Epoch [3/30]: Train Loss = 0.3675, Val Loss = 0.2560
Best model saved.
Epoch [4/30]: Train Loss = 0.3360, Val Loss = 0.2433
Best model saved.
Epoch [5/30]: Train Loss = 0.3163, Val Loss = 0.2322
Best model saved.
Epoch [6/30]: Train Loss = 0.3085, Val Loss = 0.2310
Best model saved.
Epoch [7/30]: Train Loss = 0.3077, Val Loss = 0.2290
Best model saved.
Epoch [8/30]: Train Loss = 0.3078, Val Loss = 0.2270
Best model saved.
Epoch [9/30]: Train Loss = 0.3080, Val Loss = 0.2352
Epoch [10/30]: Train Loss = 0.3072, Val Loss = 0.2304
Epoch [11/30]: Train Loss = 0.3068, Val Loss = 0.2226
Best model saved.
Epoch [12/30]: Train Loss = 0.3067, Val Loss = 0.2231
Epoch [13/30]: Train Loss = 0.3063, Val Loss = 0.2334
Epoch [14/30]: Train Loss = 0.3072, Val Loss = 0.2260
Epoch [15/30]: Trai

KeyboardInterrupt: 

In [ ]:
print("\n" + "="*20 + " Summary of Results " + "="*20)
for pred_len in pred_len_list:
    mse = results[pred_len]['Test MSE']
    mae = results[pred_len]['Test MAE']
    print(f"pred_len = {pred_len}: Test MSE = {mse:.4f}, Test MAE = {mae:.4f}")
